In [178]:
import pandas as pd
import requests
from ipywidgets import IntProgress
from IPython.display import display
import time

In [83]:
houses = pd.read_csv('../raw_data/scrapped_houses.csv')
characters = pd.read_csv('../raw_data/scrapped_characters.csv')
character_predictions = pd.read_csv('../raw_data/character-predictions.csv')
dead = pd.read_excel('../raw_data/only_dead.xlsx', header=1)

In [84]:
houses.columns # Name, Region

Index(['Name', 'Region', 'Coat of Arms', 'Words', 'Titles', 'Seats',
       'Current Lord', 'Heir', 'Overlord', 'Founded', 'Founder', 'Died Out',
       'Ancestral Weapons', 'Cadet Branches', 'Sworn Members', 'URL'],
      dtype='object')

In [85]:
houses = houses[['Name', 'Region']]

In [86]:
characters.columns # Culture, Allegiances

Index(['Name', 'Culture', 'Born', 'Died', 'Titles', 'Aliases', 'Father',
       'Mother', 'Spouse', 'Allegiances', 'Books', 'POV Books', 'TV Series',
       'Played By', 'URL'],
      dtype='object')

In [87]:
characters = characters[['Culture', 'Allegiances']]

In [88]:
character_predictions.columns # culture, house

Index(['S.No', 'actual', 'pred', 'alive', 'plod', 'name', 'title', 'male',
       'culture', 'dateOfBirth', 'DateoFdeath', 'mother', 'father', 'heir',
       'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'boolDeadRelations',
       'isPopular', 'popularity', 'isAlive'],
      dtype='object')

In [89]:
character_predictions = character_predictions[['culture', 'house']]

In [90]:
dead.columns # Allegiance

Index(['Allegiance', 'Death No.', 'Episode', 'Killer', 'Killers House',
       'Location', 'Method', 'Name', 'Season'],
      dtype='object')

In [91]:
dead = dead[['Allegiance']]

In [92]:
houses[['Region']].value_counts()

Region         
The Reach          72
The North          61
The Westerlands    55
The Riverlands     52
The Crownlands     49
Iron Islands       38
The Stormlands     38
The Vale           35
Dorne              29
The Neck            7
Beyond the Wall     1
None                1
dtype: int64

In [93]:
#houses[houses['Region'] == 'The Vale']['Name'].value_counts()

In [95]:
characters['Culture'].value_counts()

Northmen                   137
Ironborn                   127
Free Folk                   70
Valyrian                    57
Braavosi                    46
Dornish                     42
Ghiscari                    35
Dothraki                    23
Rivermen                    22
Valemen                     20
Reach                       16
Westeros                    14
Westerman                   14
Stormlands                  10
Mountain clans               9
Tyroshi                      9
Lysene                       8
First Men                    8
Northern mountain clans      8
Andal                        8
Qartheen                     7
Summer Isles                 7
Myrish                       5
Free folk                    5
Astapori                     5
Pentoshi                     4
Meereenese                   4
Crannogmen                   4
Asshai                       3
Sistermen                    3
Norvoshi                     3
Lhazareen                    3
Rhoynar 

In [128]:
mylist = []
for i in characters['Allegiances']:
    j = i.strip('[]')
    if j:
        k = j.replace("https://www.anapioficeandfire.com/api/houses/", '').\
              replace(' ', '').replace("'", '')
        if ',' in k:
            for l in k.split(','):
                mylist.append(int(l))
        else:
            mylist.append(int(k))

In [217]:
lords = pd.DataFrame(set(sorted(mylist)), columns=['house'])
lords['overlord'] = 0
lords['ol_name'] = ''

In [218]:
lords

,house,overlord,ol_name
0,2,0,
1,4,0,
2,7,0,
3,8,0,
4,10,0,
...,...,...,...
292,439,0,
293,440,0,
294,441,0,
295,443,0,


In [219]:
url = "https://www.anapioficeandfire.com/api/houses/"

In [220]:
f = IntProgress(min=0, max=len(lords)) # instantiate the bar
display(f) # display the bar

for row in range(len(lords)):
    f.value += 1
    i = lords.loc[row, 'house']
    h = requests.get(f"{url}{i}").json()
    ol = h['overlord'].strip("https://www.anapioficeandfire.com/api/houses/")
    if i == 16:
        lords.loc[row, 'overlord'] = 16
        lords.loc[row, 'ol_name'] = h['name']
    elif ol == '16':
        lords.loc[row, 'overlord'] = lords.loc[row, 'house']
        lords.loc[row, 'ol_name'] = h['name']
    else:
        lords.loc[row, 'overlord'] = ol
        if ol != '':
            over = requests.get(f"{url}{int(ol)}").json()
            lords.loc[row, 'ol_name'] = over['name']
        #print(row)

IntProgress(value=0, max=297)

In [221]:
lords[(lords['ol_name'] == 'House Yronwood of Yronwood')]

,house,overlord,ol_name
73,104,444,House Yronwood of Yronwood


In [190]:
test = requests.get(f"{url}{444}").json()

In [194]:
ol = test['overlord'].strip("https://www.anapioficeandfire.com/api/houses/")
ol

'285'

In [195]:
over = requests.get(f"{url}{int(ol)}").json()

In [196]:
over['name']

'House Nymeros Martell of Sunspear'

In [222]:
f = IntProgress(min=0, max=len(lords)) # instantiate the bar
display(f) # display the bar

for row in range(len(lords)):
    f.value += 1
    i = lords.loc[row, 'overlord']
    if i == '':
        continue
    h = requests.get(f"{url}{i}").json()
    ol = h['overlord'].strip("https://www.anapioficeandfire.com/api/houses/")
    if i == 16:
        lords.loc[row, 'overlord'] = 16
        lords.loc[row, 'ol_name'] = h['name']
    elif ol == '16':
        lords.loc[row, 'overlord'] = lords.loc[row, 'overlord']
        lords.loc[row, 'ol_name'] = h['name']
    else:
        lords.loc[row, 'overlord'] = ol
        if ol != '':
            over = requests.get(f"{url}{int(ol)}").json()
            lords.loc[row, 'ol_name'] = over['name']
        #print(row)

IntProgress(value=0, max=297)

In [223]:
row

296

In [233]:
print(lords.to_string())

     house overlord                            ol_name
0        2      285  House Nymeros Martell of Sunspear
1        4      398         House Tyrell of Highgarden
2        7        7           House Arryn of the Eyrie
3        8      398         House Tyrell of Highgarden
4       10       10         House Baelish of Harrenhal
5       11        7           House Arryn of the Eyrie
6       12      398         House Tyrell of Highgarden
7       13      229   House Lannister of Casterly Rock
8       14       15     House Baratheon of Dragonstone
9       15       15     House Baratheon of Dragonstone
10      16       16  House Baratheon of King's Landing
11      17       17     House Baratheon of Storm's End
12      18      398         House Tyrell of Highgarden
13      19        7           House Arryn of the Eyrie
14      20      229   House Lannister of Casterly Rock
15      22      398         House Tyrell of Highgarden
16      23           House Targaryen of King's Landing
17      24

In [239]:
lords[['ol_name', 'overlord']].value_counts()

ol_name                            overlord
House Tyrell of Highgarden         398         44
House Lannister of Casterly Rock   229         37
House Baelish of Harrenhal         10          27
House Greyjoy of Pyke              169         27
House Baratheon of Storm's End     17          25
House Arryn of the Eyrie           7           22
House Stark of Winterfell          362         21
House Nymeros Martell of Sunspear  285         17
House Bolton of the Dreadfort      34          16
House Baratheon of Dragonstone     15          11
                                               10
House Targaryen of King's Landing               2
House Boggs of Crackclaw Point     31           1
House Rykker of Duskendale         334          1
House Baratheon of King's Landing  16           1
House Longwaters                   243          1
House Mallery                      253          1
House Massey of Stonedance         260          1
House Nymeros Martell of Sunspear  285          1
House 

In [249]:
great_houses = lords[['ol_name']].drop_duplicates().reset_index(drop=True)

In [259]:
i = -1

In [299]:
i += 1
#great_houses.loc[i]

In [453]:
character_predictions = pd.read_csv('../raw_data/character-predictions.csv')
character_predictions = character_predictions[['culture', 'house']]

In [454]:
character_predictions['house'].value_counts()

Night's Watch      105
House Frey          97
House Stark         72
House Targaryen     62
House Lannister     49
                  ... 
House Gower          1
House Borrell        1
Citadel              1
Wise Masters         1
Three-eyed crow      1
Name: house, Length: 347, dtype: int64

In [455]:
character_predictions['culture']

0            NaN
1       Rivermen
2            NaN
3            NaN
4        Dornish
          ...   
1941    Westeros
1942         NaN
1943         NaN
1944         NaN
1945         NaN
Name: culture, Length: 1946, dtype: object

In [456]:
only_houses = character_predictions[['house']].drop_duplicates().dropna().reset_index(drop=True)

In [457]:
only_houses['house_num'] = 0
only_houses['overlord'] = ''
only_houses['overlord_num'] = 0

In [458]:
origins = []
for house in only_houses['house']:
    if 'House' not in house:
        origins.append(house)

In [459]:
i = -1

In [460]:
i += 1
#origins[i]

In [461]:
i

0

In [462]:
# for page in range(375, 379): #445
#     house = requests.get(f"{url}{page}").json()
#     print(f"\nhouse: {house['name']}")
#     for test in only_houses['house'].loc[0:6]:
#         print(test)
#         if test in house['name']:
#             print(house['url'].strip("https://www.anapioficeandfire.com/api/houses/"))

In [463]:
# for hi in only_houses['house'].loc[0:6]:
#     print(hi)

In [464]:
origins

["Night's Watch",
 'Brotherhood Without Banners',
 'Happy Port',
 'Kingswood Brotherhood',
 'Burned Men',
 'wildling',
 'Kingdom of the Three Daughters',
 'Drowned men',
 'Stormcrows',
 'Antler Men',
 'Peach',
 'Stone Crows',
 'Sea watch',
 'Second Sons',
 'Moon Brothers',
 'Faceless Men',
 'Brave Companions',
 'Band of Nine',
 "Alchemists' Guild",
 'Black Ears',
 'Faith of the Seven',
 'Pureborn',
 "R'hllor",
 'Kingsguard',
 "Chataya's brothel",
 'Brotherhood without banners',
 'Windblown',
 'Queensguard',
 "City Watch of King's Landing",
 'Blacks',
 'Citadel',
 'Wise Masters',
 'Good Masters',
 'The Citadel',
 'Iron Bank of Braavos',
 'Maesters',
 'Golden Company',
 'Company of the Cat',
 'Khal',
 'Summer Islands',
 'Undying Ones',
 'Thenn',
 'Graces',
 'Unsullied',
 'Brotherhood without Banners',
 'Mance Rayder',
 'Thirteen',
 'brotherhood without banners',
 'Three-eyed crow']

In [465]:
for i in only_houses['house']:
    print(i)

House Frey
House Swyft
House Arryn
House Santagar
House Targaryen
House Osgrey
Night's Watch
House Humble
House Wylde
House Wode
House Fell
Brotherhood Without Banners
House Webber
House Greyjoy
House Stark
House Waynwood
House Dayne
House Manderly
House Farwynd of the Lonely Light
Happy Port
House of Loraq
Kingswood Brotherhood
House Botley
Burned Men
House Velaryon
House Tallhart
House Tyrell
House Blackwood
House Blackfyre
wildling
Kingdom of the Three Daughters
House Royce of the Gates of the Moon
House Nayland
House Vance of Wayfarer's Rest
House Rowan
House Farrow
House Lonmouth
House Reyne
House Ashford
House Brax
House Paege
House Hollard
House Tarth
House Ryswell
House Lannister
House Crakehall
House Darklyn
House Westerling
House Weaver
House Connington
House Stout
House Harclay
House Drumm
House Lannister of Lannisport
House Crabb
Drowned men
Stormcrows
Antler Men
House Spicer
House Staunton
House Stonetree
House Leygood
House Bolton
House Sparr
House Fossoway of Cider Hall


In [466]:
culture = character_predictions[['culture']].drop_duplicates().dropna().reset_index(drop=True)

In [467]:
for i in culture['culture']:
    print(i)

Rivermen
Dornish
Valyrian
Ironborn
Free Folk
Northmen
Summer Isles
Braavosi
Dothraki
Ghiscari
Vale mountain clans
Reach
Tyroshi
Lhazarene
Free folk
Ironmen
Qartheen
Lysene
westermen
Westerman
Qarth
Lyseni
northmen
Qohor
Westeros
Norvoshi
First Men
Meereenese
Andal
Astapori
Westermen
ironborn
Ghiscaricari
Braavos
Stormlands
Valemen
Myrish
Lhazareen
Dornishmen
Sistermen
Northern mountain clans
Andals
Vale
Crannogmen
Wildling
Dorne
Pentoshi
free folk
Summer Islander
Westerlands
Summer Islands
Asshai
Riverlands
Naathi
Rhoynar
Meereen
Norvos
Stormlander
Wildlings
Astapor
Reachmen
Asshai'i
Ibbenese
The Reach


In [468]:
character_predictions.drop_duplicates(inplace=True)

In [469]:
character_predictions.dropna(how='all', inplace=True)

In [470]:
#character_predictions.fillna('unknown', inplace=True)

In [471]:
sort = character_predictions.sort_values(by='culture').reset_index(drop=True)

In [472]:
#sort['location'] = ''
sort['origin'] = ''

In [473]:
sort

,culture,house,origin
0,Andal,NaN,
1,Andals,NaN,
2,Asshai,R'hllor,
3,Asshai,NaN,
4,Asshai'i,NaN,
...,...,...,...
482,NaN,House Erenford,
483,NaN,House Grell,
484,NaN,brotherhood without banners,
485,NaN,R'hllor,


In [474]:
sort.to_csv('../features_for_quiz/origins.txt', index=False)

In [439]:
checking = pd.read_csv('../raw_data/character-predictions.csv')

In [440]:
checking

,S.No,actual,pred,alive,plod,name,title,male,culture,dateOfBirth,...,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,isPopular,popularity,isAlive
0,1,0,0,0.054,0.946,Viserys II Targaryen,NaN,1,NaN,NaN,...,0.0,NaN,0,0,NaN,11,1,1,0.605351,0
1,2,1,0,0.387,0.613,Walder Frey,Lord of the Crossing,1,Rivermen,208.0,...,NaN,1.0,1,1,97.0,1,1,1,0.896321,1
2,3,1,0,0.493,0.507,Addison Hill,Ser,1,NaN,NaN,...,NaN,NaN,0,1,NaN,0,0,0,0.267559,1
3,4,0,0,0.076,0.924,Aemma Arryn,Queen,0,NaN,82.0,...,NaN,0.0,1,1,23.0,0,0,0,0.183946,0
4,5,1,1,0.617,0.383,Sylva Santagar,Greenstone,0,Dornish,276.0,...,NaN,1.0,1,1,29.0,0,0,0,0.043478,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,1942,0,1,0.803,0.197,Luwin,NaN,1,Westeros,NaN,...,NaN,NaN,0,0,NaN,0,0,1,0.351171,0
1942,1943,0,1,0.987,0.013,Reek,NaN,1,NaN,NaN,...,NaN,NaN,0,0,NaN,0,0,0,0.096990,0
1943,1944,1,1,0.520,0.480,Symeon Star-Eyes,NaN,1,NaN,NaN,...,NaN,NaN,0,0,NaN,0,0,0,0.030100,1
1944,1945,1,1,0.637,0.363,Coldhands,NaN,1,NaN,NaN,...,NaN,NaN,0,0,NaN,0,0,0,0.130435,1


In [516]:
checking.loc[(checking['culture'] == 'Westeros') & (checking['house'].isna())]

,S.No,actual,pred,alive,plod,name,title,male,culture,dateOfBirth,...,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,isPopular,popularity,isAlive
635,636,0,0,0.002,0.998,Jeyne Waters,NaN,1,Westeros,171.0,...,NaN,NaN,0,0,100.0,0,0,0,0.073579,0
1907,1908,0,0,0.295,0.705,Mycah,NaN,1,Westeros,285.0,...,NaN,NaN,0,0,13.0,0,0,0,0.103679,0


In [97]:
dead.value_counts()

Allegiance                       
House Lannister                      835
None                                 432
House Stark                          121
Free Folk                            121
Night's Watch                         79
House Targaryen                       68
House Bolton                          68
Sons of the Harpy                     68
House Baratheon of Dragonstone        67
House Frey                            62
Faith Militant                        52
Smallfolk                             44
House Greyjoy                         41
Dothraki                              23
House Baratheon of King's Landing     22
The Masters                           14
The Thirteen                          12
Wise Masters                          12
Brotherhood Without Banners           10
House Martell                          8
House Mormont                          8
Good Masters                           7
House Baelish                          6
Great Masters          